In [4]:
import pandas as pd
import numpy as np
import sqlite3

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer    

In [2]:

# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("./database.sqlite")
df = pd.read_sql_query("SELECT * from Tweets", con)

# verify that result of SQL query is stored in the dataframe
con.close()

In [3]:
df.replace('',np.nan, inplace = True)
df.head(2)


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567588278875213824,neutral,1.0,NaN,NaN,Delta,NaN,JetBlueNews,NaN,0,@JetBlue's new CEO seeks the right balance to ...,NaN,2015-02-16 23:36:05 -0800,USA,Sydney
1,567590027375702016,negative,1.0,Can't Tell,0.6503,Delta,NaN,nesi_1992,NaN,0,@JetBlue is REALLY getting on my nerves !! 😡�...,NaN,2015-02-16 23:43:02 -0800,undecided,Pacific Time (US & Canada)


In [7]:
# clean text of punctuation, url, stopwords    

#stops = set(stopwords.words("english")) 


def clean_text(text):

    text = text.encode('utf-8')
    text = re.sub('https(.)*', '', text, flags=re.MULTILINE)
    text = re.sub("[^a-zA-Z]", " ",text)
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    return text
df.text = df.text.apply(lambda val: clean_text(val))

In [8]:
df.text[0:10]

0     JetBlue s new CEO seeks the right balance to ...
1     JetBlue is REALLY getting on my nerves       ...
2     united yes  We waited in line for almost an h...
3     united the we got into the gate at IAH on tim...
4     SouthwestAir its cool that my bags take a bit...
5     united and don t hope for me having a nicer f...
6     united I like delays less than you because I ...
7     united  link to current status of flights air...
8     SouthwestAir you guys there  Are we on hour  ...
9     united I tried   DM it would not go thru    n...
Name: text, dtype: object

In [10]:
from sklearn.feature_extraction.text import CountVectorizer        
from nltk.stem.porter import PorterStemmer

#######
# based on http://www.cs.duke.edu/courses/spring14/compsci290/assignments/lab02.html
stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    # remove non letters
    # tokenize
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems
######## 

In [14]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/Stav/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [33]:
print "Creating the bag of words...\n"
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = tokenize,  
                             preprocessor = None,
                             lowercase = True,
                             stop_words = 'english',   
                             max_features = 1000) 

data_features = vectorizer.fit_transform(df.text)
data_features = data_features.toarray()

vocab = vectorizer.get_feature_names()
print vocab[0:5]

Creating the bag of words...

[u'aa', u'abl', u'absolut', u'accept', u'access']


In [34]:
vocab

[u'aa',
 u'abl',
 u'absolut',
 u'accept',
 u'access',
 u'accommod',
 u'account',
 u'act',
 u'actual',
 u'ad',
 u'add',
 u'addit',
 u'address',
 u'admir',
 u'advis',
 u'advisori',
 u'afternoon',
 u'agent',
 u'ago',
 u'air',
 u'aircraft',
 u'airlin',
 u'airplan',
 u'airport',
 u'airway',
 u'alert',
 u'allow',
 u'alon',
 u'alreadi',
 u'altern',
 u'alway',
 u'amaz',
 u'america',
 u'american',
 u'americanair',
 u'americanairlin',
 u'amp',
 u'angri',
 u'ani',
 u'announc',
 u'annoy',
 u'anoth',
 u'answer',
 u'anymor',
 u'anyon',
 u'anyth',
 u'anywher',
 u'apolog',
 u'app',
 u'appar',
 u'appear',
 u'appeas',
 u'appli',
 u'appreci',
 u'area',
 u'aren',
 u'arriv',
 u'asap',
 u'ask',
 u'assign',
 u'assist',
 u'atl',
 u'atlanta',
 u'attempt',
 u'attend',
 u'attitud',
 u'au',
 u'austin',
 u'auto',
 u'autom',
 u'avail',
 u'avgeek',
 u'avoid',
 u'aw',
 u'award',
 u'away',
 u'awesom',
 u'b',
 u'babi',
 u'bad',
 u'badcustomerservic',
 u'badservic',
 u'bag',
 u'baggag',
 u'bank',
 u'base',
 u'basic',
 u

In [38]:
import numpy as np

# Sum up the counts of each vocabulary word
dist = np.sum(data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set

word_count_tuple = []
for tag, count in zip(vocab, dist):
    word_count_tuple.append((tag,count))

In [39]:
bag_words_df = pd.DataFrame(word_count_tuple)
bag_words_df.head()

,0,1
0,aa,279
1,abl,118
2,absolut,52
3,accept,54
4,access,41


In [40]:
len(data_features)

14485

In [41]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_features, df.airline_sentiment, test_size = .33)

print X_train.shape, y_train.shape, X_test.shape, y_test.shape

(9704, 1000) (9704,) (4781, 1000) (4781,)


In [52]:
print "random forest..."
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100, criterion='gini', n_jobs=-1) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit(X_train, y_train) 


y_pred = forest.predict(X_test)

print y_train.value_counts()
print y_test.value_counts()

Training the random forest...
negative    6083
neutral     2065
positive    1556
Name: airline_sentiment, dtype: int64
negative    2999
neutral     1004
positive     778
Name: airline_sentiment, dtype: int64


In [53]:
from sklearn.metrics import classification_report
cls_rep = classification_report(y_test, y_pred)
print cls_rep

             precision    recall  f1-score   support

   negative       0.81      0.89      0.85      2999
    neutral       0.57      0.46      0.51      1004
   positive       0.70      0.60      0.65       778

avg / total       0.74      0.75      0.74      4781



In [46]:
forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [12]:
print y_train.value_counts()
print y_test.value_counts()

negative    6082
neutral     2065
positive    1557
Name: airline_sentiment, dtype: int64
negative    3000
neutral     1004
positive     777
Name: airline_sentiment, dtype: int64


In [61]:
print "gradient boosting..."
from sklearn.ensemble import GradientBoostingClassifier

# Initialize a Random Forest classifier with 100 trees
forest = GradientBoostingClassifier(n_estimators = 100, criterion='gini', n_jobs=-1) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit(X_train, y_train) 


y_pred = forest.predict(X_test)

print y_train.value_counts()
print y_test.value_counts()

gradient boosting...


TypeError: __init__() got an unexpected keyword argument 'n_jobs'